---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(20)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [115]:
import re
from datetime import datetime

def date_sorter():
    
    test1 = df.str.extractall(r'(\d{1,2}[/-]+(?:[0]*[1-9]|[1-2][0-9]|3[0-1])[/-]+\d{2,4})')
    test1b = pd.to_datetime(test1[0])
    
    test2 = df.str.extractall(r'(\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[., ]+\d{2,4})')
    test2b = pd.to_datetime(test2[0])
    
    test3 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-., ]+\d{1,2}[-, ]+\d{2,4})')
    test3b = pd.to_datetime(test3[0])
    
    test4 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-., ]+\d{4})')
    test4[0] = '01 ' + test4[0].astype(str)
    test4[0] = test4[0].str.replace(r'(Decem[a-z]+ber)', 'December')
    test4[0] = test4[0].str.replace(r'Janaury', 'January')
    test4b = pd.to_datetime(test4[0])
    test4b = test4b.loc[228:342]
    
    test5 = df.str.extractall(r'(\d{1,2}[/]+\d{4})')
    test5[0] = '01/' + test5[0].astype(str)
    test5 = test5.loc[343:454]
    test5b = pd.to_datetime(test5[0], format = '%d/%m/%Y')
    
    test6 = df.str.extractall(r'(\d{4})')
    test6[0] = '01/01/' + test6[0].astype(str)
    test6 = test6.loc[455:500]
    test6b = pd.to_datetime(test6[0])
    
    dates = [test1b, test2b, test3b, test4b, test5b, test6b]
    result = pd.concat(dates)
    result.reset_index(inplace=True, drop = True)
    result = result.sort_values().reset_index()
    final = result['index']
    #result = result.rank(method = 'first').astype('int')
    #result = result - 1
    
    return final.head()

date_sorter()

0     9
1    84
2     2
3    53
4    28
Name: index, dtype: int64